In [1]:

f = open("/home/joe0400/Downloads/JDT_Bugs_sm.csv")

# clean function
import html
import re
def clean(text):
    # convert html escapes like &amp; to characters.
    text = html.unescape(text) 
    # tags like <tab>
    text = re.sub(r'<[^<>]*>', ' ', text)
    # markdown URLs like [Some text](https://....)
    text = re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', text)
    # text or code in brackets like [0]
    text = re.sub(r'\[[^\[\]]*\]', ' ', text)
    # standalone sequences of specials, matches &# but not #cool
    text = re.sub(r'(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)', ' ', text)
    # standalone sequences of hyphens like --- or ==
    text = re.sub(r'(?:^|\s)[\-=\+]{2,}(?:\s|$)', ' ', text)
    # sequences of white spaces
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

#textaciy normalization funciton
import textacy
import textacy.preprocessing as tprep
from sklearn.preprocessing import normalize as normalize_vector


if textacy.__version__ < '0.11':
    def normalize(text):
        text = tprep.normalize_hyphenated_words(text)
        text = tprep.normalize_quotation_marks(text)
        text = tprep.normalize_unicode(text)
        text = tprep.remove_accents(text)
        return text
else:
    # adjusted to textacy 0.11. Note, function names are changed
    def normalize(text):
        text = tprep.normalize.hyphenated_words(text)
        text = tprep.normalize.quotation_marks(text)
        text = tprep.normalize.unicode(text)
        text = tprep.remove.accents(text)
        return text

#spacy lemmatization using the snippet
import spacy
from nltk.corpus import wordnet
#wanted word function
def wanted_word(token):
    return wordnet.synsets(str(token))
nlp = spacy.load("en_core_web_sm", disable=["tok2vec", "tagger", "parser", "attribute_ruler","ner"])
#modified so that it returms the doc and lemmas for the dataframe
def process_using_spacy(text):
    doc = nlp(text) 
    lemmas = " ".join(token.lemma_ for token in doc if wanted_word(str(token)))
    return ' '.join([str(d) for d in doc]), lemmas

from sklearn.feature_extraction.text import TfidfVectorizer



In [2]:
import csv
# this is the csv getting loaded
table = [[process_using_spacy(normalize(clean(col)))[0 if i != 2 or j == 0 else 1] for i, col in enumerate(row) ] for j, row in enumerate(csv.reader(f))]

/home/joe0400/.local/lib/python3.8/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [3]:
#grabbing the rows and vals for reformatting the table
table_rows = table[0]
table_vals = table[1:]

vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform([v[2] for v in table_vals])
classes = {'APT', 'Core', 'Debug', 'Doc', 'Text', 'UI'}

In [4]:
# the reason for tfidfs normalization is that it makes it 
# it into cosine similarity for scikit learns implementation
tfidifs = normalize_vector(X.toarray())

In [5]:
import pandas
#making the table for pands to load
table_generated = {col:[] for col in table_rows}
table_generated['tf-idf'] = []
table_generated['id'] = []
for each in tfidifs:
    table_generated['tf-idf'].append(each)

for row in table_vals:
    for i in range(len(row)):
        table_generated[table_rows[i]].append(row[i])
    for j, each in enumerate(classes):
        if(each == row[1]):
            table_generated['id'].append(j)


In [6]:
dataframe = pandas.DataFrame(table_generated)
#pandas loading the table

In [7]:
dataframe

,,Component,bug report,tf-idf,id
0,0,APT,exception thrown reconcile got following excep...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
1,1,APT,bug getting error trying install installing ne...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
2,2,APT,unable load factory names container i have an ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
3,3,APT,headless build fails integrated external tool ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
4,4,APT,error type detection is too conservative compi...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
...,...,...,...,...,...
2638,2638,UI,trying externalize strings i got following i t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
2639,2639,UI,java search package has no plug in export see ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
2640,2640,UI,rename inner type does not update constructor ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
2641,2641,UI,disable operation logical packages in i m not ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1


In [8]:
classes = set(dataframe["Component"])

In [9]:
classes

{'APT', 'Core', 'Debug', 'Doc', 'Text', 'UI'}

In [10]:
random_class_selection = [dataframe[dataframe['id'] == classif].sample(1) for classif in set(dataframe['id'])]

In [11]:
random_choices = [(sel['tf-idf'].values[0], sel['id'].values[0]) for sel in random_class_selection]

In [13]:
from sklearn.cluster import KMeans
import numpy as np

kmeans = KMeans(verbose=1,n_clusters=6,init=np.asarray([c[0] for c in random_choices]))
tfs = np.asarray([each for each in dataframe['tf-idf']])
clusters = kmeans.fit(tfs)

/home/joe0400/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1146: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


Initialization complete
Iteration 0, inertia 4846.131650852004
Iteration 1, inertia 2376.7179736047146
Iteration 2, inertia 2297.5828871373246
Iteration 3, inertia 2291.1471951897183
Iteration 4, inertia 2285.983732660111
Iteration 5, inertia 2282.0295256041236
Iteration 6, inertia 2279.697123815059
Iteration 7, inertia 2277.7770392310013
Iteration 8, inertia 2276.4878366968915
Iteration 9, inertia 2275.5619233808648
Iteration 10, inertia 2274.871621378072
Iteration 11, inertia 2274.399348232434
Iteration 12, inertia 2273.8548787158607
Iteration 13, inertia 2273.5260450049905
Iteration 14, inertia 2273.304272175056
Iteration 15, inertia 2273.1694157458
Iteration 16, inertia 2273.074955852716
Iteration 17, inertia 2272.9837644908002
Iteration 18, inertia 2272.926037352198
Iteration 19, inertia 2272.899357508869
Iteration 20, inertia 2272.8859501129623
Iteration 21, inertia 2272.882840456005
Converged at iteration 21: strict convergence.


In [14]:
confusion_matrix = {}
for each in sorted(clusters.labels_):
    confusion_matrix[each] = [0 for b in clusters.labels_]

In [15]:
for a,b in zip(clusters.labels_,dataframe['id']):
    confusion_matrix[b][a] += 1


In [16]:
pandas.DataFrame(confusion_matrix).iloc[[0,1,2,3,4]] # to show the confusion matrix.

,0,1,2,3,4,5
0,53,103,9,1,82,36
1,141,2,0,1,6,0
2,120,392,80,161,194,251
3,7,89,6,2,171,29
4,38,95,9,10,98,48


In [ ]:
#  it seems like its not a great implementation for this.
#  Its diaganol is not great, as it often isnt a great matrix for the mix.